In [1]:
%matplotlib inline

import glob
import os
import re
import pandas as pd
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt

import skimage.transform
import scipy.ndimage
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
import tensorflow as tf

import scipy.misc
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

DATA_PATH = '/kaggle_2/luna/luna16/data/original_lungs/'
OUTPUT_FOLDER = '/kaggle_2/luna/luna16/data/pre_processed_chunks_segmented_augmented/'
ANNOTATIONS_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/annotations.csv'
ANNOTATIONS_EXCLUDED_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/annotations_excluded.csv'
CANDIDATES_PATH = '/kaggle_2/luna/luna16/data/original_lungs/csv/candidates.csv'
CHUNK_SIZE = 64
CANDIDATES_SIZE = 10000
NUM_CLASSES = 7

In [2]:
annotations = pd.read_csv(ANNOTATIONS_PATH)
annotations_excluded = pd.read_csv(ANNOTATIONS_EXCLUDED_PATH)
candidates = pd.read_csv(CANDIDATES_PATH)
candidates_sampled = candidates[candidates['class'] == 0].sample(CANDIDATES_SIZE)
candidates_sampled

,seriesuid,coordX,coordY,coordZ,class
534886,1.3.6.1.4.1.14519.5.2.1.6279.6001.922852847124...,-115.340000,69.050000,-266.630000,0
100338,1.3.6.1.4.1.14519.5.2.1.6279.6001.153985109349...,66.521215,38.706639,-50.865252,0
295199,1.3.6.1.4.1.14519.5.2.1.6279.6001.262873069163...,-68.594529,85.388926,-80.746706,0
509634,1.3.6.1.4.1.14519.5.2.1.6279.6001.776800177074...,54.340000,-11.680000,-167.930000,0
521702,1.3.6.1.4.1.14519.5.2.1.6279.6001.850739282072...,72.820000,56.330000,-58.440000,0
409182,1.3.6.1.4.1.14519.5.2.1.6279.6001.334022941831...,72.357504,-202.479660,1030.575532,0
497429,1.3.6.1.4.1.14519.5.2.1.6279.6001.733642690503...,91.625943,16.425351,-228.733784,0
268379,1.3.6.1.4.1.14519.5.2.1.6279.6001.246589849815...,-74.599595,-36.523781,-126.105025,0
11908,1.3.6.1.4.1.14519.5.2.1.6279.6001.106419850406...,36.763000,90.060298,-166.526038,0
538654,1.3.6.1.4.1.14519.5.2.1.6279.6001.943403138251...,89.190000,1.180000,-151.420000,0


In [5]:
def load_itk(filename):
    # Reads the image using SimpleITK
    itkimage = sitk.ReadImage(filename)
    
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))
    
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    
    return ct_scan, origin, spacing

In [6]:
def get_label(diameter_mm):
    if int(diameter_mm) == -1:
        return 0
    elif (diameter_mm >= 0.0) and (diameter_mm <= 15.0):
        return int(diameter_mm/3.0) + 1
    elif diameter_mm > 15.0:
        return 6

all_annotations = pd.concat([annotations_excluded, annotations], ignore_index=True)
all_annotations['class'] = all_annotations.diameter_mm.apply(get_label)
all_annotations = all_annotations.drop('diameter_mm', 1)

train_annotations = pd.concat([all_annotations, candidates_sampled], ignore_index=True)

In [7]:
train_annotations['class'].value_counts()

0    40513
2     2876
3     1511
4      587
6      528
5      315
1       48
Name: class, dtype: int64

In [8]:
def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

In [9]:
def largest_label_volume(im, bg=-1):
    vals, counts = np.unique(im, return_counts=True)

    counts = counts[vals != bg]
    vals = vals[vals != bg]

    if len(counts) > 0:
        return vals[np.argmax(counts)]
    else:
        return None

def segment_lung_mask(image, fill_lung_structures=True):
    
    # not actually binary, but 1 and 2. 
    # 0 is treated as background, which we do not want
    binary_image = np.array(image > -320, dtype=np.int8)+1
    labels = measure.label(binary_image)
    
    # Pick the pixel in the very corner to determine which label is air.
    #   Improvement: Pick multiple background labels from around the patient
    #   More resistant to "trays" on which the patient lays cutting the air 
    #   around the person in half
    background_label = labels[0,0,0]
    
    #Fill the air around the person
    binary_image[background_label == labels] = 2
    
    
    # Method of filling the lung structures (that is superior to something like 
    # morphological closing)
    if fill_lung_structures:
        # For every slice we determine the largest solid structure
        for i, axial_slice in enumerate(binary_image):
            axial_slice = axial_slice - 1
            labeling = measure.label(axial_slice)
            l_max = largest_label_volume(labeling, bg=0)
            
            if l_max is not None: #This slice contains some lung
                binary_image[i][labeling != l_max] = 1

    
    binary_image -= 1 #Make the image actual binary
    binary_image = 1-binary_image # Invert it, lungs are now 1
    
    # Remove other air pockets insided body
    labels = measure.label(binary_image, background=0)
    l_max = largest_label_volume(labels, bg=0)
    if l_max is not None: # There are air pockets
        binary_image[labels != l_max] = 0
 
    return binary_image

In [10]:
def plot_2d(im):
    # Show some slice in the middle
    plt.imshow(im, cmap=plt.cm.bone)
    plt.show()
    
def plot_3d(image, threshold=-300):
    # Position the scan upright, 
    # so the head of the patient would be at the top facing the camera
    p = image.transpose(2,1,0)
    
    verts, faces = measure.marching_cubes(p, threshold)

    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Fancy indexing: `verts[faces]` to generate a collection of triangles
    mesh = Poly3DCollection(verts[faces], alpha=0.1)
    face_color = [0.5, 0.5, 1]
    mesh.set_facecolor(face_color)
    ax.add_collection3d(mesh)

    ax.set_xlim(0, p.shape[0])
    ax.set_ylim(0, p.shape[1])
    ax.set_zlim(0, p.shape[2])

    plt.show()

In [11]:
patient_uids = train_annotations.seriesuid.unique()

patients_processed_files = glob.glob(OUTPUT_FOLDER + '[0-9\.]*_X.npy')
patients_processed = set()
for filename in patients_processed_files:
    m = re.match(r'([0-9\.]*)_X.npy', os.path.basename(filename))
    patients_processed.add(m.group(1))

print(len(patient_uids))

graph = tf.Graph()
with graph.as_default():
    img_chunk = tf.placeholder(tf.float32, shape=[64, 64, 64], name='img_chunk')
    img_flipped_up_down = tf.image.flip_up_down(img_chunk)
    img_flipped_left_right = tf.image.flip_left_right(img_chunk)
    img_rot_90 = tf.image.rot90(img_chunk)
    img_trans = tf.image.transpose_image(img_chunk)

for patient_uid in patient_uids[0:1]:
    if patient_uid in patients_processed:
        print('Skipping already processed patient {}'.format(patient_uid))
        continue
    print('Processing patient {}'.format(patient_uid))
    
    patient_annotations = train_annotations[train_annotations.seriesuid == patient_uid]
    patient_scans_path = glob.glob(DATA_PATH + 'subset?/{}.mhd'.format(patient_uid))[0]
    img, origin, spacing = load_itk(patient_scans_path)

    #calculate resize factor
    RESIZE_SPACING = [1, 1, 1]
    resize_factor = spacing / RESIZE_SPACING
    new_real_shape = img.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize = new_shape / img.shape
    new_spacing = spacing / real_resize

    # Resample; resize image to 1mmx1mmx1mm spacing
    lung_img = scipy.ndimage.interpolation.zoom(img, real_resize) 

    # Segment
    lung_img_segment_mask = segment_lung_mask(lung_img, True)   

    lung_img = np.multiply(lung_img, lung_img_segment_mask)
    
    num_class_not_0 = patient_annotations[patient_annotations['class'] != 0].shape[0]
    num_class_0 = patient_annotations.shape[0] - num_class_not_0
      
    count = 0
    X = np.ndarray([num_class_0 + num_class_not_0 * 5, 64, 64, 64], dtype=np.int16)
    Y = np.ndarray([num_class_0 + num_class_not_0 * 5, NUM_CLASSES], dtype=np.int16)
    for annotation in patient_annotations.itertuples():
        y = annotation[5]
        coordX = annotation[2]
        coordY = annotation[3]
        coordZ = annotation[4]
        imageCoord = np.array((coordZ, coordY, coordX))

        # Convert coords to voxel coords and slice lung_img
        imageCoord = world_2_voxel(imageCoord, origin, new_spacing)
        
        coordX1 = int(imageCoord[2] - (CHUNK_SIZE/2))
        coordX2 = int(imageCoord[2] + (CHUNK_SIZE/2))
        coordY1 = int(imageCoord[1] - (CHUNK_SIZE/2))
        coordY2 = int(imageCoord[1] + (CHUNK_SIZE/2))
        coordZ1 = int(imageCoord[0] - (CHUNK_SIZE/2))
        coordZ2 = int(imageCoord[0] + (CHUNK_SIZE/2))
        
        coordX1 = 0 if (coordX1 < 0) else coordX1
        coordY1 = 0 if (coordY1 < 0) else coordY1
        coordZ1 = 0 if (coordZ1 < 0) else coordZ1
        
        coordX2 = lung_img.shape[2] if (coordX2 > lung_img.shape[2]) else coordX2
        coordY2 = lung_img.shape[1] if (coordY2 > lung_img.shape[1]) else coordY2
        coordZ2 = lung_img.shape[0] if (coordZ2 > lung_img.shape[0]) else coordZ2
      
        chunk = np.full((64, 64, 64), -1000.0, np.float32)
        chunk[0:coordZ2-coordZ1, 0:coordY2-coordY1, 0:coordX2-coordX1] = lung_img[coordZ1:coordZ2,coordY1:coordY2,coordX1:coordX2]

        X[count,:,:,:] = chunk
        Y[count,] = (np.arange(NUM_CLASSES) == y)+0
        count = count + 1
        
        # Augment non-zero classes (cancerous nodules)
        if y != 0:
            with tf.Session(graph=graph) as session:
                session.run(tf.global_variables_initializer())
                chunk_flipped_up_down, chunk_flipped_left_right, chunk_rot_90, chunk_trans = session.run([img_flipped_up_down, 
                                                                                                          img_flipped_left_right,
                                                                                                          img_rot_90,
                                                                                                          img_trans], feed_dict={img_chunk: chunk})
                
                X[count,:,:,:] = chunk_flipped_up_down
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_flipped_left_right
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_rot_90
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1

                X[count,:,:,:] = chunk_trans
                Y[count,] = (np.arange(NUM_CLASSES) == y)+0
                count = count + 1
  
    np.save(OUTPUT_FOLDER + patient_uid + '_X.npy', X)
    np.save(OUTPUT_FOLDER + patient_uid + '_Y.npy', Y)

888
Processing patient 1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222365663678666836860


KeyboardInterrupt: 